# Внешние отраслевые источники

## https://b1.ru/b1-surveys

In [1]:
import re
import time
import hashlib

import requests
from bs4 import BeautifulSoup


def get_soup(link):
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }
    soup = None
    response = requests.get(link, headers=HEADERS)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
    else:
        print(f"Ошибка запроса: {response.status_code}")
    return soup


URL = 'https://b1.ru/'
SURVEYS_URL = URL + 'b1-surveys/'


soup = get_soup(SURVEYS_URL)
pagination = soup.find('div', class_='pagination')
page_links = [a['href'] for a in pagination.find_all('a') if '?PAGEN_2=' in a['href']]
last_page = max([int(a.split('?PAGEN_2=')[-1]) for a in page_links])

In [3]:
analytics = []
for i in range(1, last_page + 1):
    soup = get_soup(SURVEYS_URL + '?PAGEN_2=' + str(i))
    time.sleep(1)
    analytics_block = {}
    for block in soup.find_all('div', class_=re.compile(r"^news-block__gridItem triangleWrap--hover")):
        if block:
            link = block['src-href']
            if not 'services' in link:
                analytics_block['link'] = link
                analytics_block['date'] = block.find('p', class_='news-block__date').text
                analytics_block['name'] = block.find('h2', class_='news-block__title').text
                analytics_block['name_en'] = link.strip().strip('/').split('/')[-1]
                
                time.sleep(1)
                analytics_soup = get_soup(URL + link)
                tags_block = analytics_soup.find('div', class_=re.compile(r"tezis-list-block__tags"))
                if tags_block:
                    analytics_block['tags'] = [a.text for a in tags_block.find_all('a', class_=re.compile(r"tag"))]
                else:
                    analytics_block['tags'] = []
                
                analytics_block['text'] = '\n'.join([re.sub(r"\n+", "\n", x.text) for x in analytics_soup.find('main').find('section').find_next_siblings()])
                analytics_block['pdf_links'] = [f'{URL}{a['href']}' for a in analytics_soup.find_all('a', href=True) if a['href'].endswith('.pdf')]
                analytics_block['pdf_links'].append(f'{URL}local/assets/surveys/{analytics_block['name_en']}.pdf')
                
                pdfs_content = []
                pdfs_names = []
                for url in analytics_block['pdf_links']:
                    response = requests.get(url, stream=True)
                    if response.status_code == 200:
                        pdf_name = url.strip().strip('/').split('/')[-1]
                        pdf_content = response.content
                        if pdf_name not in pdfs_names:
                            pdfs_names.append(pdf_name)
                            pdfs_content.append({"name": pdf_name, "content": pdf_content})
                analytics_block['pdfs'] = pdfs_content
                analytics.append(analytics_block)
        break
    break

In [4]:
len(analytics)

1